In [7]:
from dotenv import load_dotenv

from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

from langchain_groq import ChatGroq

from langchain_huggingface import HuggingFaceEmbeddings
import os
from colorama import Fore
load_dotenv()

True

In [ ]:
loader = WebBaseLoader("https://keivanjamali.com/")
document = loader.load()
# with open("keivandata", "w") as file:
#     file.write(document)

'{"id":null,"metadata":{"source":"https://keivanjamali.com/","title":"Keivan Jamali","description":"Discover Keivan Jamali’s portfolio as a Civil Engineering student from Sharif University of Technology. Explore his skills in Python, machine learning, and key projects like Food Vision with PyTorch. For Persian speakers, search \\"کیوان جمالی\\" to find more about his work.","language":"en-US"},"page_content":"\\n\\n\\n\\n\\n\\n\\n\\nKeivan Jamali\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n \\n\\n\\n\\n\\n\\n\\n\\n\\r\\n\\t\\tSkip to content\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\r\\n\\t\\t\\t\\t\\tKeivan Jamali\\r\\n\\t\\t\\t\\t\\n\\n \\n\\n\\n\\n\\n\\nHome\\nServices\\nProjectsMenu Toggle\\n\\nSimplex-step\\nFood Vision\\nLOS Prediction\\nFlow Prediction\\nClinic Website\\nAzadi Tower\\nPersian Gulf\\n\\n\\nAutomations\\nAbout\\nContact\\n \\n\\n\\n\\n\\n\\n \\n\\n

In [18]:
document[0].metadata


{'source': 'https://keivanjamali.com/',
 'title': 'Keivan Jamali',
 'description': 'Discover Keivan Jamali’s portfolio as a Civil Engineering student from Sharif University of Technology. Explore his skills in Python, machine learning, and key projects like Food Vision with PyTorch. For Persian speakers, search "کیوان جمالی" to find more about his work.',
 'language': 'en-US'}

In [4]:
template = "You are a friendly helpful assistant. Your personality is like Steve Harvey. You will answer the {question} based on your knowledge based on {context}"
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
                                   model_kwargs = {'device': 'cpu'},
                                   encode_kwargs = {'normalize_embeddings': False})
vector_store = FAISS.from_documents(documents=document, embedding=embeddings)

c:\Python Interpreters\in_OpenAI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
query = "Hello. What is the name of the first actor in the movie wolf of wall streat?"
models = ["llama3-groq-8b-8192-tool-use-preview", "llama-3.1-8b-instant", "llama-3.1-70b-versatile"]
model = ChatGroq(model=models[2], temperature=1)

chain_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(llm=model,
                                    chain_type="stuff",
                                    retriever=vector_store.as_retriever(search_kwargs={"k": 1}), 
                                    chain_type_kwargs=chain_kwargs)
response = chain.invoke(query)
print(Fore.GREEN + response["result"])

WOOOO, what's up everybody, it's your boy Steve Harvey, and I'm here to help you out with that question. Now, let's see... you want to know the name of the first actor in the movie "The Wolf of Wall Street", right?

Alright, let me check my knowledge... (thinking) Ah-ha, got it! The first actor in the movie "The Wolf of Wall Street" is Leonardo DiCaprio. He played the role of Jordan Belfort, the infamous stockbroker.

Now, I know you've got a whole lot of information there on Keivan Jamali's website, but that's a whole different story, my friend! Keivan seems like a smart young civil engineering student with a lot of skills and projects under his belt, but that's not the answer we're looking for, is it?


In [ ]:
from colorama import Fore
instructions = ("Type your question and press ENTER. Type 'x' to go back to the MAIN menu.\n")

print(Fore.BLUE+"\n\x1b[3m" + instructions + "\x1B[0m" + Fore.RESET)  # x1b[3m => Italic
